In [2]:
import torch
from torch import nn
import numpy as np
from mapsgan import Trajectories
import sys
%load_ext autoreload
%autoreload 2

In [4]:
td = Trajectories('/mnt/Clouds/MapsGAN/data/eth/test')

In [361]:
model = nn.RNN(input_size=input_size, hidden_size=hidden_size, batch_first = True)
criterion = nn.CrossEntropyLoss()
optim = torch.optim.Adam(model.parameters(), lr = 0.1)
hidden = torch.zeros(num_layers, batch_size, hidden_size)

for epoch in range(100):
    optim.zero_grad()
    loss = 0
    output, hidden = model(x_onehot, hidden)
    output = output.view(sequence_length, -1)
    #print([idx2char(int(torch.max(hid, 1)[1])] for hid in hidden])
    loss += criterion(output, labels.squeeze().long())
    
#     for input, label in zip(x_onehot, labels):
#         hidden, output = model(input, hidden)
#         _, pred = torch.max(output, 1)
#         print(idx2char[int(pred)])
#         loss  += criterion(output, label.long())
        
    print("epoch: %d, loss: %1.3f"%(epoch+1, loss.data[0]))
    loss.backward(retain_graph=True)
    optim.step()


RuntimeError: Assertion `cur_target >= 0 && cur_target < n_classes' failed.  at /opt/conda/conda-bld/pytorch-cpu_1532578932944/work/aten/src/THNN/generic/ClassNLLCriterion.c:93

## Trajectory Toy Example

In [146]:
class ToyGenerator(nn.Module):
    
    def __init__(self, input_dim = 32, hidden_dim = 20, batch_size = 32):
        super().__init__()
        self.batch_size = batch_size
        self.hidden_dim = hidden_dim
        self.input_dim = input_dim
        self.lstm = nn.LSTM(input_dim, hidden_dim)
        self.linear = nn.Sequential(nn.Linear(hidden_dim, 20*2),
                                    nn.ReLU())
        
    def init_hidden(self):
        return torch.zeros(1, self.batch_size, self.hidden_dim)
        
    def forward(self, z):
        _, (x, _) = self.lstm(z)
        x = self.linear(x)
        return x.view(self.hidden_dim, self.batch_size, 2)

In [201]:
class Discriminator(nn.Module):
    def __init__(self, input_dim=2, mlp_dim=16, hidden_dim=64, num_layers=1):
        super(Discriminator, self).__init__()
        self.input_dim = input_dim
        self.mlp_dim = mlp_dim
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        
        self.mlp_spatial_lstm = nn.Sequential(#nn.Linear(input_dim, mlp_dim),
                                              #nn.ReLU(),
                                              nn.LSTM(#mlp_dim,
                                                      input_dim,
                                                      hidden_dim,
                                                      num_layers))
        self.mlp_bool = nn.Sequential(nn.Linear(hidden_dim, 20),
                                      nn.Sigmoid())
        
    def forward(self, input_seq):
        _, (hidden, _) = self.mlp_spatial_lstm(input_seq)
        return self.mlp_bool(hidden.squeeze()).view(20, 32, 1)

In [202]:
TGen = ToyGenerator()
z = torch.randn(32, 32, 32)
TGen(z).shape

torch.Size([20, 32, 2])

In [225]:
TGen = ToyGenerator()
D = Discriminator()
ganloss_G = nn.BCELoss()
l1loss_G = nn.L1Loss()
ganloss_D = nn.BCELoss()
optimG = torch.optim.Adam(TGen.parameters(), lr = 0.1)
optimD = torch.optim.Adam(D.parameters(), lr = 0.01)
epochs = 200
loss_G_l1_list = []
loss_G_gan_list = []
loss_D_list = []
for epoch in range(epochs):
    data_idx = 1#np.random.randint(len(td)) # get random sample
    labels = td[data_idx]['lossmask'] # get lossmask (20, 32)
    groundtruth = torch.cat((td[data_idx]['train'], td[data_idx]['groundtruth']), dim = 0)  # groundtruth (20, 32, 2)
    
    '''Train generator'''
    loss_G = 0
    optimG.zero_grad()
    z = torch.randn(32, 32, 32)
    prediction = TGen(z)
    out_D_pred = D(prediction)
    loss_G_l1 = l1loss_G(prediction[:,  0:td.peds_per_seq[data_idx]], groundtruth[:,  0:td.peds_per_seq[data_idx]]) # l1loss
    loss_G_l1_list.append(loss_G_l1)
    loss_G_gan = ganloss_G(out_D_pred.squeeze()[:, 0:td.peds_per_seq[data_idx]], labels[:, 0:td.peds_per_seq[data_idx]]) # ganloss
    loss_G_gan_list.append(loss_G_gan)
    loss_G = loss_G_l1 + loss_G_gan
    loss_G.backward(retain_graph=True)
    optimG.step()

    '''Train discriminator'''
    optimD.zero_grad()
    loss_D = 0
    out_D_real = D(seq.unsqueeze(0))
    out_D_fake = D(prediction)
    loss_D += ganloss_D(out_D_real.squeeze()[:, 0:td.peds_per_seq[data_idx]], labels[:, 0:td.peds_per_seq[data_idx]]) # on real
    loss_D += ganloss_D(out_D_fake.squeeze()[:, 0:td.peds_per_seq[data_idx]], ((labels-1)*(-1))[:, 0:td.peds_per_seq[data_idx]]) # on fake
    loss_D_list.append(loss_D)
    loss_D.backward(retain_graph=True)
    optimD.step()

    print("epoch: %d, loss_G_total: %1.3f, loss_G_l1: %1.3f, loss_G_gan: %1.3f, loss_D: %1.3f"%(epoch+1, loss_G, loss_G_l1, loss_G_gan, loss_D))  

epoch: 1, loss_G_total: 8.284, loss_G_l1: 7.577, loss_G_gan: 0.707, loss_D: 1.386
epoch: 2, loss_G_total: 8.188, loss_G_l1: 7.455, loss_G_gan: 0.733, loss_D: 1.363
epoch: 3, loss_G_total: 7.792, loss_G_l1: 7.016, loss_G_gan: 0.776, loss_D: 1.327
epoch: 4, loss_G_total: 7.204, loss_G_l1: 6.261, loss_G_gan: 0.942, loss_D: 1.210
epoch: 5, loss_G_total: 6.323, loss_G_l1: 4.868, loss_G_gan: 1.455, loss_D: 0.988
epoch: 6, loss_G_total: 5.483, loss_G_l1: 3.468, loss_G_gan: 2.014, loss_D: 0.872
epoch: 7, loss_G_total: 5.296, loss_G_l1: 2.678, loss_G_gan: 2.618, loss_D: 0.810
epoch: 8, loss_G_total: 5.668, loss_G_l1: 2.439, loss_G_gan: 3.229, loss_D: 0.776
epoch: 9, loss_G_total: 6.169, loss_G_l1: 2.400, loss_G_gan: 3.768, loss_D: 0.756
epoch: 10, loss_G_total: 6.556, loss_G_l1: 2.342, loss_G_gan: 4.215, loss_D: 0.740
epoch: 11, loss_G_total: 6.803, loss_G_l1: 2.223, loss_G_gan: 4.580, loss_D: 0.723
epoch: 12, loss_G_total: 7.068, loss_G_l1: 2.193, loss_G_gan: 4.875, loss_D: 0.704
epoch: 13, lo

In [227]:
z = torch.randn(32, 32, 32)
predicted = TGen(z)

In [231]:
predicted[0, 0:td.peds_per_seq[data_idx]], groundtruth[0,0:td.peds_per_seq[data_idx]]

(tensor([[10.9467,  6.0442],
         [ 9.7658,  6.3908]], grad_fn=<SliceBackward>),
 tensor([[12.5100,  6.1900],
         [12.0900,  6.9500]]))